In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00


In [2]:
# Bernstein–Vazirani Algorithm using Qiskit 2.x
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle(qc, inputs, ancilla, s):
    """Implements oracle for f(x) = s · x (no constant b)."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    fig = plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most

if __name__ == '__main__':
    s = '1011'
    print('Secret string s =', s)
    qc = bernstein_vazirani_circuit(s)
    print(qc.draw(fold=-1))
    measured = run_bv(qc)
    if measured == s:
        print('✅ Successfully recovered secret string s')
    else:
        print('⚠️ Measured string differs from s (noise or error).')


Secret string s = 1011
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101
⚠️ Measured string differs from s (noise or error).


# Task
Modify the selected empty cell "8ZLuqdD17dc9" to change the secret string `s` in the Bernstein-Vazirani algorithm, verify the measured output matches the new `s`, modify the oracle to include an additional constant bit `b` (i.e., implement `f(x) = s·x ⊕ b`) and show how `b` affects the ancilla only, add noise via `qiskit_aer.noise.NoiseModel` and analyze robustness, and create a notebook that explains each step with visualizations and markdown.

## Modify the oracle

### Subtask:
Update the `bv_oracle` function to include an additional constant bit `b` (i.e., implement `f(x) = s·x ⊕ b`).


**Reasoning**:
Modify the `bv_oracle` function to accept an additional parameter `b` and apply an X gate to the ancilla if `b` is '1'.



In [3]:
def bv_oracle(qc, inputs, ancilla, s, b):
    """Implements oracle for f(x) = s · x (no constant b)."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    if b == '1':
        qc.x(ancilla)

## Analyze the effect of b

### Subtask:
Show how the constant bit `b` affects only the ancilla qubit's final state and doesn't change the measured output of the input qubits.


**Reasoning**:
Modify the circuit function to accept the constant bit b, then generate and print circuits for both b='0' and b='1' to visually compare the effect of b on the ancilla qubit.



In [4]:
def bernstein_vazirani_circuit(s, b='0'):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    qc.barrier() # Add barrier for visual separation
    bv_oracle(qc, inputs, ancilla, s, b)
    qc.barrier() # Add barrier for visual separation
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

s = '1011'
print('Secret string s =', s)

print("\nCircuit for b = '0':")
qc_b0 = bernstein_vazirani_circuit(s, b='0')
print(qc_b0.draw(fold=-1))

print("\nCircuit for b = '1':")
qc_b1 = bernstein_vazirani_circuit(s, b='1')
print(qc_b1.draw(fold=-1))

Secret string s = 1011

Circuit for b = '0':
     ┌───┐      ░                 ░ ┌───┐┌─┐         
q_0: ┤ H ├──────░───■─────────────░─┤ H ├┤M├─────────
     ├───┤      ░   │             ░ ├───┤└╥┘┌─┐      
q_1: ┤ H ├──────░───┼─────────────░─┤ H ├─╫─┤M├──────
     ├───┤      ░   │             ░ ├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├──────░───┼────■────────░─┤ H ├─╫──╫─┤M├───
     ├───┤      ░   │    │        ░ ├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├──────░───┼────┼────■───░─┤ H ├─╫──╫──╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌─┴─┐┌─┴─┐ ░ └───┘ ║  ║  ║ └╥┘
q_4: ┤ X ├┤ H ├─░─┤ X ├┤ X ├┤ X ├─░───────╫──╫──╫──╫─
     └───┘└───┘ ░ └───┘└───┘└───┘ ░       ║  ║  ║  ║ 
c: 4/═════════════════════════════════════╩══╩══╩══╩═
                                          0  1  2  3 

Circuit for b = '1':
     ┌───┐      ░                      ░ ┌───┐┌─┐         
q_0: ┤ H ├──────░───■──────────────────░─┤ H ├┤M├─────────
     ├───┤      ░   │                  ░ ├───┤└╥┘┌─┐      
q_1: ┤ H ├──────░───┼──────────────────░─┤ H ├─╫─┤M├──

**Reasoning**:
Run the circuits for both cases (b='0' and b='1') using the run_bv function and compare the measured outputs of the input qubits.



In [5]:
print("\nRunning circuit for b = '0':")
measured_b0 = run_bv(qc_b0)

print("\nRunning circuit for b = '1':")
measured_b1 = run_bv(qc_b1)

print(f"\nMeasured output for b='0': {measured_b0}")
print(f"Measured output for b='1': {measured_b1}")


Running circuit for b = '0':
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101

Running circuit for b = '1':
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101

Measured output for b='0': 1101
Measured output for b='1': 1101


## Add noise model

### Subtask:
Introduce a noise model using `qiskit_aer.noise.NoiseModel` to the `AerSimulator` and analyze its effect on the results.


**Reasoning**:
Introduce a noise model using `qiskit_aer.noise.NoiseModel` to the `AerSimulator` and analyze its effect on the results by running the circuit and plotting the counts.



In [6]:
# 1. Import the NoiseModel class from qiskit_aer.noise.
from qiskit_aer.noise import NoiseModel, depolarizing_error

# 2. Create a simple noise model.
# Define error probabilities for single-qubit and two-qubit gates
p1q = 0.001  # 0.1% error probability for single-qubit gates
p2q = 0.01   # 1% error probability for two-qubit gates

# Create depolarizing error channels
error_1 = depolarizing_error(p1q, 1)
error_2 = depolarizing_error(p2q, 2)

# Create a noise model
noise_model = NoiseModel()

# Add errors to the noise model
# Add single-qubit errors to all single-qubit gates
noise_model.add_all_qubit_quantum_error(error_1, ['h', 'x'])

# Add two-qubit errors to all two-qubit gates
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

print("Noise Model created:")
print(noise_model)

# 3. Instantiate AerSimulator with the created noise model.
sim_noise = AerSimulator(noise_model=noise_model)

# 4. Run the bernstein_vazirani_circuit with the secret string s = '1011' and b = '0'
s = '1011'
b = '0'
print(f"\nRunning circuit with secret string s = '{s}' and constant bit b = '{b}' under noise:")
qc = bernstein_vazirani_circuit(s, b=b)

# Transpile the circuit for the noisy simulator
tqc_noise = transpile(qc, sim_noise)

# Run the simulation
shots = 1024
job_noise = sim_noise.run(tqc_noise, shots=shots)
result_noise = job_noise.result()

# 5. Get the counts from the simulation result.
counts_noise = result_noise.get_counts()

# 6. Print the counts and observe how the noise affects the distribution.
print('\nNoisy Counts:', counts_noise)

# 7. Use plot_histogram to visualize the noisy results.
print("\nHistogram of noisy results:")
fig_noise = plot_histogram(counts_noise, title="Bernstein-Vazirani with Noise")
plt.show()

# For comparison, let's also run the noiseless simulation again and plot it
print("\nRunning noiseless simulation for comparison:")
sim_noiseless = AerSimulator()
tqc_noiseless = transpile(qc, sim_noiseless)
job_noiseless = sim_noiseless.run(tqc_noiseless, shots=shots)
result_noiseless = job_noiseless.result()
counts_noiseless = result_noiseless.get_counts()

print('\nNoiseless Counts:', counts_noiseless)
print("\nHistogram of noiseless results:")
fig_noiseless = plot_histogram(counts_noiseless, title="Bernstein-Vazirani Noiseless")
plt.show()

# Analyze robustness by comparing the most frequent outcome
most_noisy = max(counts_noise, key=counts_noise.get)
most_noiseless = max(counts_noiseless, key=counts_noiseless.get)

print('\nMost frequent measured bitstring (noisy):', most_noisy)
print('Most frequent measured bitstring (noiseless):', most_noiseless)

if most_noisy == s:
    print('✅ Most frequent noisy measurement matches the secret string s')
else:
    print('⚠️ Most frequent noisy measurement differs from s (due to noise).')

if most_noiseless == s:
    print('✅ Noiseless measurement matches the secret string s')
else:
    print('⚠️ Noiseless measurement differs from s (unexpected).')


Noise Model created:
NoiseModel:
  Basis gates: ['cx', 'h', 'id', 'rz', 'sx', 'x']
  Instructions with noise: ['h', 'cx', 'x']
  All-qubits errors: ['h', 'x', 'cx']

Running circuit with secret string s = '1011' and constant bit b = '0' under noise:

Noisy Counts: {'0000': 4, '1000': 1, '1001': 6, '0001': 6, '1111': 2, '0101': 6, '0111': 1, '1101': 998}

Histogram of noisy results:

Running noiseless simulation for comparison:

Noiseless Counts: {'1101': 1024}

Histogram of noiseless results:

Most frequent measured bitstring (noisy): 1101
Most frequent measured bitstring (noiseless): 1101
⚠️ Most frequent noisy measurement differs from s (due to noise).
⚠️ Noiseless measurement differs from s (unexpected).


**Reasoning**:
The previous execution showed that the most frequent noisy and noiseless measurements do not match the secret string '1011', instead they match '1101'. This indicates an issue with the current setup of the Bernstein-Vazirani circuit or the secret string variable `s`. It is important to fix this discrepancy before concluding on the effect of noise. The noiseless case should ideally yield only the correct secret string. Let's re-examine the secret string `s` and the circuit logic.



In [7]:
# Let's double-check the secret string 's' and the expected outcome.
# The Bernstein-Vazirani algorithm should output the secret string 's'.
# In the previous runs, with s = '1011', the measured output was consistently '1101'.
# This suggests that either '1011' is not the 's' being encoded or there is a
# misunderstanding of the circuit output mapping.

# Let's test with a simpler secret string, e.g., s = '01'.
s_test = '01'
b_test = '0'
print(f"\nTesting with s = '{s_test}' and b = '{b_test}' (noiseless):")
qc_test = bernstein_vazirani_circuit(s_test, b=b_test)
print(qc_test.draw(fold=-1))

sim_test = AerSimulator()
tqc_test = transpile(qc_test, sim_test)
job_test = sim_test.run(tqc_test, shots=1024)
result_test = job_test.result()
counts_test = result_test.get_counts()

print('\nTest Counts (noiseless):', counts_test)
fig_test = plot_histogram(counts_test, title=f"Bernstein-Vazirani (s='{s_test}', noiseless)")
plt.show()

most_test = max(counts_test, key=counts_test.get)
print('Most frequent measured bitstring (test):', most_test)

if most_test == s_test:
    print(f'✅ Noiseless measurement matches the test secret string s = {s_test}')
else:
    print(f'⚠️ Noiseless measurement differs from the test secret string s = {s_test}.')

# Re-running with original s = '1011' and b = '0' to confirm the behavior
s_original = '1011'
b_original = '0'
print(f"\nRe-running with original s = '{s_original}' and b = '{b_original}' (noiseless):")
qc_original = bernstein_vazirani_circuit(s_original, b=b_original)
sim_original = AerSimulator()
tqc_original = transpile(qc_original, sim_original)
job_original = sim_original.run(tqc_original, shots=1024)
result_original = job_original.result()
counts_original = result_original.get_counts()

print('\nOriginal Counts (noiseless):', counts_original)
fig_original = plot_histogram(counts_original, title=f"Bernstein-Vazirani (s='{s_original}', noiseless)")
plt.show()

most_original = max(counts_original, key=counts_original.get)
print('Most frequent measured bitstring (original):', most_original)

if most_original == s_original:
    print(f'✅ Noiseless measurement matches the original secret string s = {s_original}')
else:
    print(f'⚠️ Noiseless measurement differs from the original secret string s = {s_original}.')



Testing with s = '01' and b = '0' (noiseless):
     ┌───┐      ░       ░ ┌───┐┌─┐   
q_0: ┤ H ├──────░───────░─┤ H ├┤M├───
     ├───┤      ░       ░ ├───┤└╥┘┌─┐
q_1: ┤ H ├──────░───■───░─┤ H ├─╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐ ░ └───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├─░─┤ X ├─░───────╫──╫─
     └───┘└───┘ ░ └───┘ ░       ║  ║ 
c: 2/═══════════════════════════╩══╩═
                                0  1 

Test Counts (noiseless): {'10': 1024}
Most frequent measured bitstring (test): 10
⚠️ Noiseless measurement differs from the test secret string s = 01.

Re-running with original s = '1011' and b = '0' (noiseless):

Original Counts (noiseless): {'1101': 1024}
Most frequent measured bitstring (original): 1101
⚠️ Noiseless measurement differs from the original secret string s = 1011.


**Reasoning**:
The noiseless simulations for both '01' and '1011' secret strings yielded incorrect outputs ('10' instead of '01' and '1101' instead of '1011'). This indicates a fundamental issue with the circuit implementation or the interpretation of the measured output bits. Before analyzing noise, it's crucial to fix the circuit to correctly identify the secret string in the noiseless case. The issue is likely in the measurement mapping or how the secret string bits are ordered. Let's re-examine the code to ensure the secret string bits `s` are correctly mapped to the input qubits and the measurement results are read in the corresponding order. The Qiskit documentation states that measurement maps qubits to classical bits in the order specified. The secret string is applied to qubits q_0 to q_(n-1), and these are measured into classical bits c_0 to c_(n-1). The secret string `s` is given as a string where `s[0]` corresponds to the least significant bit of the input `x`. However, in the `bv_oracle` function, `s[i]` is applied to `inputs[i]`, where `inputs` is `list(range(n))`, meaning `inputs[0]` is q_0, `inputs[1]` is q_1, and so on. When measuring `inputs` into `creg`, the measurement outcome is read as a string where the rightmost bit corresponds to the measurement of the last qubit in the `inputs` list (q_(n-1)), and the leftmost bit corresponds to the measurement of the first qubit (q_0). Therefore, the measured bitstring is in reverse order of the secret string `s` as it is currently defined (from left to right). To fix this, we can either reverse the secret string when applying it in the oracle or reverse the measured bitstring before comparing it to `s`. Reversing the measured bitstring seems more straightforward to align with the provided secret string format.



In [8]:
# Re-running the noiseless simulation with the original secret string s = '1011' and b = '0'.
# This time, we will reverse the measured bitstring for comparison with the secret string.
s_original = '1011'
b_original = '0'
print(f"\nRe-running with original s = '{s_original}' and b = '{b_original}' (noiseless), reversing measured output:")
qc_original = bernstein_vazirani_circuit(s_original, b=b_original)
sim_original = AerSimulator()
tqc_original = transpile(qc_original, sim_original)
job_original = sim_original.run(tqc_original, shots=1024)
result_original = job_original.result()
counts_original = result_original.get_counts()

print('\nOriginal Counts (noiseless):', counts_original)
fig_original = plot_histogram(counts_original, title=f"Bernstein-Vazirani (s='{s_original}', noiseless)")
plt.show()

most_original = max(counts_original, key=counts_original.get)
# Reverse the measured bitstring for comparison
most_original_reversed = most_original[::-1]

print('Most frequent measured bitstring (original):', most_original)
print('Most frequent measured bitstring (original, reversed):', most_original_reversed)


if most_original_reversed == s_original:
    print(f'✅ Noiseless measurement (reversed) matches the original secret string s = {s_original}')
else:
    print(f'⚠️ Noiseless measurement (reversed) differs from the original secret string s = {s_original}.')

# Now, let's proceed with the noise model analysis again, using the corrected interpretation.

# 1. Import the NoiseModel class from qiskit_aer.noise. (Already imported)
# from qiskit_aer.noise import NoiseModel, depolarizing_error

# 2. Create a simple noise model. (Already defined p1q, p2q and noise_model)
# Define error probabilities for single-qubit and two-qubit gates
# p1q = 0.001  # 0.1% error probability for single-qubit gates
# p2q = 0.01   # 1% error probability for two-qubit gates

# Create depolarizing error channels
# error_1 = depolarizing_error(p1q, 1)
# error_2 = depolarizing_error(p2q, 2)

# Create a noise model
# noise_model = NoiseModel()

# Add errors to the noise model
# noise_model.add_all_qubit_quantum_error(error_1, ['h', 'x'])
# noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

print("\nUsing the previously created Noise Model:")
print(noise_model)

# 3. Instantiate AerSimulator with the created noise model.
sim_noise = AerSimulator(noise_model=noise_model)

# 4. Run the bernstein_vazirani_circuit with s = '1011' and b = '0' using the noisy simulator.
s = '1011'
b = '0'
print(f"\nRunning circuit with secret string s = '{s}' and constant bit b = '{b}' under noise:")
qc = bernstein_vazirani_circuit(s, b=b)

# Transpile the circuit for the noisy simulator
tqc_noise = transpile(qc, sim_noise)

# Run the simulation
shots = 1024
job_noise = sim_noise.run(tqc_noise, shots=shots)
result_noise = job_noise.result()

# 5. Get the counts from the simulation result.
counts_noise = result_noise.get_counts()

# 6. Print the counts and observe how the noise affects the distribution.
print('\nNoisy Counts:', counts_noise)

# 7. Use plot_histogram to visualize the noisy results.
print("\nHistogram of noisy results:")
fig_noise = plot_histogram(counts_noise, title="Bernstein-Vazirani with Noise")
plt.show()

# Analyze robustness by comparing the most frequent outcome (reversed)
most_noisy = max(counts_noise, key=counts_noise.get)
most_noisy_reversed = most_noisy[::-1]

print('Most frequent measured bitstring (noisy):', most_noisy)
print('Most frequent measured bitstring (noisy, reversed):', most_noisy_reversed)

if most_noisy_reversed == s:
    print('✅ Most frequent noisy measurement (reversed) matches the secret string s')
else:
    print('⚠️ Most frequent noisy measurement (reversed) differs from s (due to noise).')



Re-running with original s = '1011' and b = '0' (noiseless), reversing measured output:

Original Counts (noiseless): {'1101': 1024}
Most frequent measured bitstring (original): 1101
Most frequent measured bitstring (original, reversed): 1011
✅ Noiseless measurement (reversed) matches the original secret string s = 1011

Using the previously created Noise Model:
NoiseModel:
  Basis gates: ['cx', 'h', 'id', 'rz', 'sx', 'x']
  Instructions with noise: ['h', 'cx', 'x']
  All-qubits errors: ['h', 'x', 'cx']

Running circuit with secret string s = '1011' and constant bit b = '0' under noise:

Noisy Counts: {'0001': 1, '0101': 6, '0000': 3, '1100': 3, '1001': 5, '1101': 1006}

Histogram of noisy results:
Most frequent measured bitstring (noisy): 1101
Most frequent measured bitstring (noisy, reversed): 1011
✅ Most frequent noisy measurement (reversed) matches the secret string s


## Change secret string s

### Subtask:
Modify the secret string `s` in the main part of the script and verify that the measured output still matches the new `s` when running on the simulator without noise.


**Reasoning**:
I need to modify the secret string 's' in the main part of the script and verify that the measured output (reversed) matches the new 's' when running on the noiseless simulator. I will update the 's' variable, run the circuit with the new 's' and b='0', simulate it using AerSimulator, get the counts, find the most frequent result, reverse it, and compare it to the new 's'.



In [9]:
# 1. Choose a new secret string s of the same length as the original ('1011').
new_s = '0110'
print('New secret string s =', new_s)

# 2. Update the value of the s variable in the main execution block of the script.
# This is done by using the new_s variable directly in the function call below.

# 3. Keep the constant bit b set to '0' for this verification step.
b = '0'
print('Constant bit b =', b)

# 4. Run the bernstein_vazirani_circuit function with the new s and b='0'.
qc = bernstein_vazirani_circuit(new_s, b=b)
print("\nCircuit for s =", new_s, "and b =", b)
print(qc.draw(fold=-1))

# 5. Use the noiseless AerSimulator to run the circuit and get the measurement counts.
sim = AerSimulator()
tqc = transpile(qc, sim)

# 6. Run the simulation
shots = 1024
job = sim.run(tqc, shots=shots)
result = job.result()

# 7. Get the counts from the simulation result.
counts = result.get_counts()
print('\nCounts:', counts)

# 8. Print the counts and plot the histogram to visualize the results.
print("\nHistogram of results:")
fig = plot_histogram(counts, title=f"Bernstein-Vazirani (s='{new_s}', noiseless)")
plt.show()

# 9. Identify the most frequent measured bitstring from the counts.
most_frequent_measured = max(counts, key=counts.get)
print('Most frequent measured bitstring:', most_frequent_measured)

# 10. Reverse the most frequent measured bitstring.
most_frequent_measured_reversed = most_frequent_measured[::-1]
print('Most frequent measured bitstring (reversed):', most_frequent_measured_reversed)

# 11. Compare the reversed measured bitstring with the new secret string s to verify they match.
# 12. Print a message indicating whether the measured string (reversed) successfully matches the new secret string.
if most_frequent_measured_reversed == new_s:
    print(f'✅ Measured string (reversed) matches the new secret string s = {new_s}')
else:
    print(f'⚠️ Measured string (reversed) differs from the new secret string s = {new_s}.')

New secret string s = 0110
Constant bit b = 0

Circuit for s = 0110 and b = 0
     ┌───┐      ░            ░ ┌───┐┌─┐         
q_0: ┤ H ├──────░────────────░─┤ H ├┤M├─────────
     ├───┤      ░            ░ ├───┤└╥┘┌─┐      
q_1: ┤ H ├──────░───■────────░─┤ H ├─╫─┤M├──────
     ├───┤      ░   │        ░ ├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├──────░───┼────■───░─┤ H ├─╫──╫─┤M├───
     ├───┤      ░   │    │   ░ ├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├──────░───┼────┼───░─┤ H ├─╫──╫──╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌─┴─┐ ░ └───┘ ║  ║  ║ └╥┘
q_4: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────╫──╫──╫──╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ║  ║  ║  ║ 
c: 4/════════════════════════════════╩══╩══╩══╩═
                                     0  1  2  3 

Counts: {'0110': 1024}

Histogram of results:
Most frequent measured bitstring: 0110
Most frequent measured bitstring (reversed): 0110
✅ Measured string (reversed) matches the new secret string s = 0110


## Add explanations and visualizations

### Subtask:
Create markdown cells to explain each step of the Bernstein-Vazirani algorithm, including the modified oracle, the effect of `b`, and the impact of noise. Use visualizations (like circuit drawings and histograms) to illustrate the concepts and results.


**Reasoning**:
The subtask requires creating markdown cells to explain the Bernstein-Vazirani algorithm, the oracle modifications, the effect of 'b', the introduction of noise, and the interpretation of results, supported by visualizations already present in the notebook. I will create the necessary markdown cells following the instructions.



In [10]:
from IPython.display import display, Markdown

# 1. Introduction to Bernstein-Vazirani Algorithm
display(Markdown("""
## The Bernstein-Vazirani Algorithm

The Bernstein-Vazirani algorithm is a celebrated quantum algorithm that demonstrates a clear separation between quantum and classical computation. It solves the problem of identifying a secret binary string $s$ encoded in a specific function $f(x)$ with fewer queries than any classical algorithm.

The function $f(x)$ is promised to be of the form $f(x) = s \cdot x$, where $x$ is a binary input string of the same length as $s$, and $s \cdot x$ is the bitwise dot product modulo 2. That is, $s \cdot x = s_0x_0 \oplus s_1x_1 \oplus \dots \oplus s_{n-1}x_{n-1}$, where $\oplus$ denotes addition modulo 2.

The goal of the algorithm is to find the secret string $s$ using a quantum computer. A classical algorithm would need to query the function $f(x)$ $n$ times (where $n$ is the length of $s$) to determine each bit of $s$. The Bernstein-Vazirani algorithm can find $s$ with just one query to a quantum oracle implementing $f(x)$.

The algorithm proceeds in the following steps:
1. Initialize a quantum state.
2. Apply Hadamard gates to the input register.
3. Query the quantum oracle that computes $f(x)$.
4. Apply Hadamard gates to the input register again.
5. Measure the input register.

The measured state of the input register directly reveals the secret string $s$.
"""))

# 2. Explanation of the Oracle (f(x) = s · x)
display(Markdown("""
## The Bernstein-Vazirani Oracle ($f(x) = s \cdot x$)

The core of the Bernstein-Vazirani algorithm is the quantum oracle, which implements the function $f(x) = s \cdot x$ in a quantum mechanical way. The oracle takes an input state $|x\rangle|y\rangle$ and transforms it to $|x\rangle|y \oplus f(x)\rangle$.

For the Bernstein-Vazirani algorithm, the oracle is designed to flip the ancilla qubit (initially in the state $|-\rangle$) based on the value of $f(x)$. The transformation is $|x\rangle|-\rangle \rightarrow |x\rangle|f(x) \oplus -\rangle = |x\rangle|f(x) \cdot -\rangle$. Since $|-\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$, $y \oplus f(x)$ in the exponent of $-1$ results in a phase kickback on the input register. Specifically, $|x\rangle|-\rangle \rightarrow (-1)^{f(x)}|x\rangle|-\rangle$.

The oracle for $f(x) = s \cdot x$ is constructed using CNOT gates. For each bit $s_i$ in the secret string $s$:
- If $s_i = 1$, a CNOT gate is applied with the $i$-th input qubit as the control and the ancilla qubit as the target. This flips the ancilla if the $i$-th input bit is 1.
- If $s_i = 0$, no operation is performed on the $i$-th input qubit regarding the ancilla.

The combined effect of these CNOT gates is that the ancilla qubit is flipped if and only if the bitwise dot product $s \cdot x$ is 1.

The following code cell defines the `bv_oracle` function that implements this logic.
"""))

# 3. Explanation of the Modified Oracle (f(x) = s · x ⊕ b)
display(Markdown("""
## Modifying the Oracle to Include a Constant Bit $b$ ($f(x) = s \cdot x \oplus b$)

The standard Bernstein-Vazirani problem considers functions of the form $f(x) = s \cdot x$. We can extend this to include a constant bit $b$, so the function becomes $f(x) = s \cdot x \oplus b$, where $\oplus$ is addition modulo 2.

The quantum oracle for this modified function takes the input state $|x\rangle|y\rangle$ and transforms it to $|x\rangle|y \oplus (s \cdot x \oplus b)\rangle$.

Implementing the $s \cdot x$ part is done as before, using CNOT gates controlled by the input qubits based on the bits of $s$. To include the $\oplus b$ term, we need an additional operation that flips the ancilla qubit if $b=1$, regardless of the input $x$. This is achieved by applying an X gate (NOT gate) to the ancilla qubit if $b$ is '1'.

The modified `bv_oracle` function incorporates this: after applying the CNOT gates for the $s \cdot x$ part, it checks the value of $b$. If $b$ is '1', it applies an X gate to the ancilla.

The following code cell defines the modified `bv_oracle` function.
"""))

# 4. Analyzing the Effect of b
display(Markdown("""
## The Effect of the Constant Bit $b$

The Bernstein-Vazirani algorithm is designed to reveal the secret string $s$. The constant bit $b$ in the function $f(x) = s \cdot x \oplus b$ affects the function's output but should not change the measured outcome of the input register which encodes $s$.

Let's analyze the oracle's action with the ancilla in the $|-\rangle$ state:
$|x\rangle|-\rangle \rightarrow |x\rangle| (s \cdot x \oplus b) \oplus -\rangle = |x\rangle| (s \cdot x \oplus b) \cdot -\rangle = (-1)^{s \cdot x \oplus b} |x\rangle|-\rangle = (-1)^{s \cdot x} (-1)^b |x\rangle|-\rangle$.

The phase kickback on the input register becomes $(-1)^{s \cdot x} (-1)^b$. The term $(-1)^b$ is a global phase factor on the input register (since it's the same for all $|x\rangle$) and does not affect the measurement probabilities of the input qubits after the final Hadamard gates. It only affects the phase of the ancilla qubit's final state.

Therefore, changing the value of $b$ from '0' to '1' should only change the final state of the ancilla qubit (by applying an extra phase) but should not alter the measurement results of the input qubits, which will still reveal $s$.

The following code will construct and run the Bernstein-Vazirani circuit for the same secret string $s$ but with $b='0'$ and $b='1'$. We will observe the circuit diagrams and the measured outputs of the input register to confirm that the measured $s$ remains the same, while the circuit structure shows the additional operation on the ancilla when $b='1'$.
"""))

# 6. Explanation of Noise Model
display(Markdown(f"""
## Introducing Noise to the Quantum Circuit

Real-world quantum computers are subject to various sources of noise and errors. These imperfections can cause qubits to flip their states unintentionally, introduce unwanted phase shifts, and generally lead to deviations from the ideal, noiseless computation. Simulating noise is crucial for understanding how algorithms perform on actual quantum hardware and for developing error mitigation or correction techniques.

In this section, we introduce a simple noise model to the `AerSimulator` to simulate the effect of errors on the Bernstein-Vazirani algorithm. We will use depolarizing errors, which are a common model for noise where a qubit's state is transformed into a completely mixed state with some probability.

We define error probabilities for single-qubit gates (`h`, `x`) and two-qubit gates (`cx`).
- `p1q = {p1q}`: Probability of a single-qubit depolarizing error occurring after a single-qubit gate.
- `p2q = {p2q}`: Probability of a two-qubit depolarizing error occurring after a two-qubit gate.

A `NoiseModel` object from `qiskit_aer.noise` is created, and these depolarizing errors are added to the model, specifying which gate types are affected. The `AerSimulator` is then instantiated with this `noise_model`.

Running the quantum circuit on this noisy simulator will produce results that are not perfectly the ideal outcome. Instead of seeing only the correct secret string $s$ with 100% probability (as in the noiseless case), we expect to see the correct string with the highest probability, but also other bitstrings with lower probabilities due to errors occurring during the computation.

The following code block sets up the noise model, runs the Bernstein-Vazirani circuit with this noise model, and displays the resulting counts and histogram. We will compare this to the noiseless results to see the impact of noise.
"""))

# 7. Interpretation of Noisy Results
display(Markdown("""
## Interpreting the Results with Noise

The histogram of the noisy simulation results clearly shows the impact of errors. Unlike the ideal noiseless case where only the secret string $s$ is measured with high probability (ideally 100%), the noisy simulation produces a distribution of outcomes.

The most frequent outcome in the noisy histogram should ideally still correspond to the secret string $s$ (after reversing the bitstring due to the measurement convention). However, other bitstrings appear with non-zero counts. These incorrect outcomes are a direct consequence of the simulated noise. Depolarizing errors occurring during the execution of the quantum gates can cause qubits to flip or their states to become mixed, leading to erroneous measurements.

Comparing the noisy histogram to the noiseless one visually demonstrates the effect of noise. The noiseless histogram will show a single sharp peak at the correct secret string, while the noisy histogram will show a peak at the correct string but also smaller bars for incorrect strings, indicating errors.

The presence of these incorrect outcomes highlights the challenge of performing quantum computations on real hardware. Error mitigation and error correction techniques are necessary to obtain reliable results from noisy quantum computers.
"""))

# 8. Changing the Secret String (Noiseless Verification)
display(Markdown("""
## Verifying the Algorithm with a Different Secret String (Noiseless)

To further demonstrate the correctness of the Bernstein-Vazirani algorithm and our implementation, we will change the secret string $s$ and run the circuit again in the noiseless environment. The algorithm should successfully identify this new secret string.

This step serves as a verification that the circuit and the oracle correctly encode and reveal arbitrary secret strings of the given length, provided there is no noise.

We will choose a new secret string, construct the circuit with this new $s$ (keeping $b='0'$ for simplicity in this verification), run it on the noiseless simulator, and check if the measured output (reversed) matches the new $s$.

The following code block performs this verification step.
"""))

# Ensure visualizations from previous steps are displayed
# This is handled by the Jupyter notebook environment automatically when code cells are executed.

<>:9: SyntaxWarning: invalid escape sequence '\c'
<>:25: SyntaxWarning: invalid escape sequence '\c'
<>:42: SyntaxWarning: invalid escape sequence '\c'
<>:59: SyntaxWarning: invalid escape sequence '\c'
<>:9: SyntaxWarning: invalid escape sequence '\c'
<>:25: SyntaxWarning: invalid escape sequence '\c'
<>:42: SyntaxWarning: invalid escape sequence '\c'
<>:59: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipython-input-377145834.py:9: SyntaxWarning: invalid escape sequence '\c'
  The function $f(x)$ is promised to be of the form $f(x) = s \cdot x$, where $x$ is a binary input string of the same length as $s$, and $s \cdot x$ is the bitwise dot product modulo 2. That is, $s \cdot x = s_0x_0 \oplus s_1x_1 \oplus \dots \oplus s_{n-1}x_{n-1}$, where $\oplus$ denotes addition modulo 2.
/tmp/ipython-input-377145834.py:25: SyntaxWarning: invalid escape sequence '\c'
  ## The Bernstein-Vazirani Oracle ($f(x) = s \cdot x$)
/tmp/ipython-input-377145834.py:42: SyntaxWarning: invalid escape sequ


## The Bernstein-Vazirani Algorithm

The Bernstein-Vazirani algorithm is a celebrated quantum algorithm that demonstrates a clear separation between quantum and classical computation. It solves the problem of identifying a secret binary string $s$ encoded in a specific function $f(x)$ with fewer queries than any classical algorithm.

The function $f(x)$ is promised to be of the form $f(x) = s \cdot x$, where $x$ is a binary input string of the same length as $s$, and $s \cdot x$ is the bitwise dot product modulo 2. That is, $s \cdot x = s_0x_0 \oplus s_1x_1 \oplus \dots \oplus s_{n-1}x_{n-1}$, where $\oplus$ denotes addition modulo 2.

The goal of the algorithm is to find the secret string $s$ using a quantum computer. A classical algorithm would need to query the function $f(x)$ $n$ times (where $n$ is the length of $s$) to determine each bit of $s$. The Bernstein-Vazirani algorithm can find $s$ with just one query to a quantum oracle implementing $f(x)$.

The algorithm proceeds in the following steps:
1. Initialize a quantum state.
2. Apply Hadamard gates to the input register.
3. Query the quantum oracle that computes $f(x)$.
4. Apply Hadamard gates to the input register again.
5. Measure the input register.

The measured state of the input register directly reveals the secret string $s$.



## The Bernstein-Vazirani Oracle ($f(x) = s \cdot x$)

The core of the Bernstein-Vazirani algorithm is the quantum oracle, which implements the function $f(x) = s \cdot x$ in a quantum mechanical way. The oracle takes an input state $|xangle|yangle$ and transforms it to $|xangle|y \oplus f(x)angle$.

For the Bernstein-Vazirani algorithm, the oracle is designed to flip the ancilla qubit (initially in the state $|-angle$) based on the value of $f(x)$. The transformation is $|xangle|-angle ightarrow |xangle|f(x) \oplus -angle = |xangle|f(x) \cdot -angle$. Since $|-angle = rac{1}{\sqrt{2}}(|0angle - |1angle)$, $y \oplus f(x)$ in the exponent of $-1$ results in a phase kickback on the input register. Specifically, $|xangle|-angle ightarrow (-1)^{f(x)}|xangle|-angle$.

The oracle for $f(x) = s \cdot x$ is constructed using CNOT gates. For each bit $s_i$ in the secret string $s$:
- If $s_i = 1$, a CNOT gate is applied with the $i$-th input qubit as the control and the ancilla qubit as the target. This flips the ancilla if the $i$-th input bit is 1.
- If $s_i = 0$, no operation is performed on the $i$-th input qubit regarding the ancilla.

The combined effect of these CNOT gates is that the ancilla qubit is flipped if and only if the bitwise dot product $s \cdot x$ is 1.

The following code cell defines the `bv_oracle` function that implements this logic.



## Modifying the Oracle to Include a Constant Bit $b$ ($f(x) = s \cdot x \oplus b$)

The standard Bernstein-Vazirani problem considers functions of the form $f(x) = s \cdot x$. We can extend this to include a constant bit $b$, so the function becomes $f(x) = s \cdot x \oplus b$, where $\oplus$ is addition modulo 2.

The quantum oracle for this modified function takes the input state $|xangle|yangle$ and transforms it to $|xangle|y \oplus (s \cdot x \oplus b)angle$.

Implementing the $s \cdot x$ part is done as before, using CNOT gates controlled by the input qubits based on the bits of $s$. To include the $\oplus b$ term, we need an additional operation that flips the ancilla qubit if $b=1$, regardless of the input $x$. This is achieved by applying an X gate (NOT gate) to the ancilla qubit if $b$ is '1'.

The modified `bv_oracle` function incorporates this: after applying the CNOT gates for the $s \cdot x$ part, it checks the value of $b$. If $b$ is '1', it applies an X gate to the ancilla.

The following code cell defines the modified `bv_oracle` function.



## The Effect of the Constant Bit $b$

The Bernstein-Vazirani algorithm is designed to reveal the secret string $s$. The constant bit $b$ in the function $f(x) = s \cdot x \oplus b$ affects the function's output but should not change the measured outcome of the input register which encodes $s$.

Let's analyze the oracle's action with the ancilla in the $|-angle$ state:
$|xangle|-angle ightarrow |xangle| (s \cdot x \oplus b) \oplus -angle = |xangle| (s \cdot x \oplus b) \cdot -angle = (-1)^{s \cdot x \oplus b} |xangle|-angle = (-1)^{s \cdot x} (-1)^b |xangle|-angle$.

The phase kickback on the input register becomes $(-1)^{s \cdot x} (-1)^b$. The term $(-1)^b$ is a global phase factor on the input register (since it's the same for all $|xangle$) and does not affect the measurement probabilities of the input qubits after the final Hadamard gates. It only affects the phase of the ancilla qubit's final state.

Therefore, changing the value of $b$ from '0' to '1' should only change the final state of the ancilla qubit (by applying an extra phase) but should not alter the measurement results of the input qubits, which will still reveal $s$.

The following code will construct and run the Bernstein-Vazirani circuit for the same secret string $s$ but with $b='0'$ and $b='1'$. We will observe the circuit diagrams and the measured outputs of the input register to confirm that the measured $s$ remains the same, while the circuit structure shows the additional operation on the ancilla when $b='1'$.



## Introducing Noise to the Quantum Circuit

Real-world quantum computers are subject to various sources of noise and errors. These imperfections can cause qubits to flip their states unintentionally, introduce unwanted phase shifts, and generally lead to deviations from the ideal, noiseless computation. Simulating noise is crucial for understanding how algorithms perform on actual quantum hardware and for developing error mitigation or correction techniques.

In this section, we introduce a simple noise model to the `AerSimulator` to simulate the effect of errors on the Bernstein-Vazirani algorithm. We will use depolarizing errors, which are a common model for noise where a qubit's state is transformed into a completely mixed state with some probability.

We define error probabilities for single-qubit gates (`h`, `x`) and two-qubit gates (`cx`).
- `p1q = 0.001`: Probability of a single-qubit depolarizing error occurring after a single-qubit gate.
- `p2q = 0.01`: Probability of a two-qubit depolarizing error occurring after a two-qubit gate.

A `NoiseModel` object from `qiskit_aer.noise` is created, and these depolarizing errors are added to the model, specifying which gate types are affected. The `AerSimulator` is then instantiated with this `noise_model`.

Running the quantum circuit on this noisy simulator will produce results that are not perfectly the ideal outcome. Instead of seeing only the correct secret string $s$ with 100% probability (as in the noiseless case), we expect to see the correct string with the highest probability, but also other bitstrings with lower probabilities due to errors occurring during the computation.

The following code block sets up the noise model, runs the Bernstein-Vazirani circuit with this noise model, and displays the resulting counts and histogram. We will compare this to the noiseless results to see the impact of noise.



## Interpreting the Results with Noise

The histogram of the noisy simulation results clearly shows the impact of errors. Unlike the ideal noiseless case where only the secret string $s$ is measured with high probability (ideally 100%), the noisy simulation produces a distribution of outcomes.

The most frequent outcome in the noisy histogram should ideally still correspond to the secret string $s$ (after reversing the bitstring due to the measurement convention). However, other bitstrings appear with non-zero counts. These incorrect outcomes are a direct consequence of the simulated noise. Depolarizing errors occurring during the execution of the quantum gates can cause qubits to flip or their states to become mixed, leading to erroneous measurements.

Comparing the noisy histogram to the noiseless one visually demonstrates the effect of noise. The noiseless histogram will show a single sharp peak at the correct secret string, while the noisy histogram will show a peak at the correct string but also smaller bars for incorrect strings, indicating errors.

The presence of these incorrect outcomes highlights the challenge of performing quantum computations on real hardware. Error mitigation and error correction techniques are necessary to obtain reliable results from noisy quantum computers.



## Verifying the Algorithm with a Different Secret String (Noiseless)

To further demonstrate the correctness of the Bernstein-Vazirani algorithm and our implementation, we will change the secret string $s$ and run the circuit again in the noiseless environment. The algorithm should successfully identify this new secret string.

This step serves as a verification that the circuit and the oracle correctly encode and reveal arbitrary secret strings of the given length, provided there is no noise.

We will choose a new secret string, construct the circuit with this new $s$ (keeping $b='0'$ for simplicity in this verification), run it on the noiseless simulator, and check if the measured output (reversed) matches the new $s$.

The following code block performs this verification step.


## Summarize and conclude

### Subtask:
Add a markdown cell to summarize the findings and conclude the notebook.


**Reasoning**:
Add a markdown cell to summarize the findings and conclude the notebook as per the instructions.



In [11]:
from IPython.display import display, Markdown

display(Markdown("""
## Summary and Conclusion

This notebook explored the Bernstein-Vazirani algorithm, a foundational example showcasing quantum speedup. We implemented the algorithm using Qiskit, demonstrating its ability to identify a secret binary string $s$ with a single quantum query.

We began by implementing the standard Bernstein-Vazirani oracle for functions of the form $f(x) = s \cdot x$. We then modified the oracle to handle functions including a constant bit, $f(x) = s \cdot x \oplus b$. Through circuit diagrams and simulation results, we verified that while the constant bit $b$ influences the state of the ancilla qubit, it **does not affect the measured outcome of the input register**, which correctly reveals the secret string $s$. This confirmed our theoretical analysis of how $b$ only introduces a global phase factor on the input register.

We also investigated the impact of noise on the algorithm's performance by introducing a simple depolarizing noise model using `qiskit_aer.noise`. Running the circuit on a noisy simulator showed that while the most frequent outcome still corresponded to the secret string $s$ (after reversing the measured bitstring), the presence of noise led to a distribution of measurement outcomes, with incorrect strings appearing with non-zero probabilities. This highlighted the susceptibility of quantum algorithms to environmental noise and the importance of considering noise in practical quantum computation.

Finally, we verified the algorithm's correctness in the noiseless case by changing the secret string $s$ and confirming that the measured output (reversed) consistently matched the new secret string.

The Bernstein-Vazirani algorithm provides a clear and intuitive demonstration of how quantum computers can outperform classical computers for specific problems. While our simulations with noise illustrate the challenges of implementing quantum algorithms on real hardware, they also underscore the importance of ongoing research into error mitigation and fault-tolerant quantum computing. The ability of the algorithm to extract the secret string $s$ in a single query, even in the presence of some noise (depending on the noise levels), remains a powerful illustration of quantum advantage.
"""))

<>:8: SyntaxWarning: invalid escape sequence '\c'
<>:8: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipython-input-3176401431.py:8: SyntaxWarning: invalid escape sequence '\c'
  We began by implementing the standard Bernstein-Vazirani oracle for functions of the form $f(x) = s \cdot x$. We then modified the oracle to handle functions including a constant bit, $f(x) = s \cdot x \oplus b$. Through circuit diagrams and simulation results, we verified that while the constant bit $b$ influences the state of the ancilla qubit, it **does not affect the measured outcome of the input register**, which correctly reveals the secret string $s$. This confirmed our theoretical analysis of how $b$ only introduces a global phase factor on the input register.



## Summary and Conclusion

This notebook explored the Bernstein-Vazirani algorithm, a foundational example showcasing quantum speedup. We implemented the algorithm using Qiskit, demonstrating its ability to identify a secret binary string $s$ with a single quantum query.

We began by implementing the standard Bernstein-Vazirani oracle for functions of the form $f(x) = s \cdot x$. We then modified the oracle to handle functions including a constant bit, $f(x) = s \cdot x \oplus b$. Through circuit diagrams and simulation results, we verified that while the constant bit $b$ influences the state of the ancilla qubit, it **does not affect the measured outcome of the input register**, which correctly reveals the secret string $s$. This confirmed our theoretical analysis of how $b$ only introduces a global phase factor on the input register.

We also investigated the impact of noise on the algorithm's performance by introducing a simple depolarizing noise model using `qiskit_aer.noise`. Running the circuit on a noisy simulator showed that while the most frequent outcome still corresponded to the secret string $s$ (after reversing the measured bitstring), the presence of noise led to a distribution of measurement outcomes, with incorrect strings appearing with non-zero probabilities. This highlighted the susceptibility of quantum algorithms to environmental noise and the importance of considering noise in practical quantum computation.

Finally, we verified the algorithm's correctness in the noiseless case by changing the secret string $s$ and confirming that the measured output (reversed) consistently matched the new secret string.

The Bernstein-Vazirani algorithm provides a clear and intuitive demonstration of how quantum computers can outperform classical computers for specific problems. While our simulations with noise illustrate the challenges of implementing quantum algorithms on real hardware, they also underscore the importance of ongoing research into error mitigation and fault-tolerant quantum computing. The ability of the algorithm to extract the secret string $s$ in a single query, even in the presence of some noise (depending on the noise levels), remains a powerful illustration of quantum advantage.


## Summary:

### Data Analysis Key Findings

*   The `bv_oracle` function was successfully modified to implement $f(x) = s \cdot x \oplus b$ by adding an X gate on the ancilla qubit conditioned on the value of $b$.
*   The constant bit $b$ affects only the ancilla qubit's state (by introducing a global phase) and does not change the measured output of the input qubits, which correctly reveals the secret string $s$.
*   Introducing a depolarizing noise model causes the measurement outcomes to deviate from the ideal single result, showing a distribution of bitstrings, although the most frequent outcome (when reversed) still corresponds to the secret string for the noise levels used.
*   The Bernstein-Vazirani algorithm implementation correctly identifies the secret string $s$ in the noiseless case, even when the secret string is changed.

### Insights or Next Steps

*   Investigate the effect of different types and levels of noise on the algorithm's success probability to understand its robustness limits.
*   Explore error mitigation techniques to improve the reliability of the Bernstein-Vazirani algorithm when run on noisy quantum simulators or hardware.
